In [5]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import re

from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
#load original SDC to construct dataframe and create new identifiers
sdc = pd.read_excel("C:/Users/47523/Downloads/2019 Summer RA/SDC 2001-2015 AT 05-28-19.xls")
sdc.head()

,deal_number,anndate,effdate,sample,permno_acq,permco_acq,cik_acq,ncusip_acq,ncusip6_acq,comnam_acq,conm_acq,acq_name,permno_tar,permco_tar,cik_tar,ncusip_tar,ncusip6_tar,comnam_tar,conm_tar,tar_name
0,1051503020,2001-07-02,2001-12-01,NaN,88373,7257,751364.0,20221810,202218,COMMERCIAL NET LEASE REALTY INC,NATIONAL RETAIL PROPERTIES,Commercial Net Lease Rlty Inc,85537,15827,1045281.0,14072410,140724,CAPTEC NET LEASE REALTY INC,CAPTEC NET LEASE REALTY INC,Captec Net Lease Realty Inc
1,1081496020,2001-01-10,2001-05-24,NaN,87447,36383,1090727.0,91131210,911312,UNITED PARCEL SERVICE INC,UNITED PARCEL SERVICE INC,UPS,77989,11750,890662.0,35884610,358846,FRITZ COMPANIES INC,FRITZ COS INC,Fritz Cos Inc
2,1082845020,2001-01-16,2001-03-20,NaN,79991,12667,913949.0,55610010,556100,MACROMEDIA INC,MACROMEDIA INC,Macromedia Inc,86565,16383,1016139.0,01671410,016714,ALLAIRE CORP,ALLAIRE CORP,Allaire Corp
3,1084648020,2001-01-22,2001-03-15,NaN,52038,21698,96021.0,87182910,871829,SYSCO CORP,SYSCO CORP,Sysco Corp,40345,6271,722642.0,40163010,401630,GUEST SUPPLY INC,GUEST SUPPLY INC,Guest Supply Inc
4,1084702020,2001-01-23,2001-07-02,NaN,10725,9662,819692.0,16090310,160903,CHARTER ONE FINANCIAL INC,CHARTER ONE FINANCIAL INC,"Charter One Finl,Cleveland,OH",77764,11605,885638.0,01852J10,01852J,ALLIANCE BANCORP INC,ALLIANCE BANCORP INC,Alliance Bancorp


In [7]:
import datetime
sdc['anndate_3yrsago'] = sdc['anndate'] - datetime.timedelta(days=3*365)

In [8]:
identifiers = sdc[['anndate','anndate_3yrsago','tar_name']]
identifiers['tar_name']=identifiers['tar_name'].str.lower()
identifiers.sample(5)

,anndate,anndate_3yrsago,tar_name
718,2006-07-30,2003-07-31,m-systems flash disk pioneers
1248,2015-02-12,2012-02-13,comscore inc
301,2003-03-07,2000-03-07,computer motion inc
866,2007-10-29,2004-10-29,arizona star resource corp
271,2002-11-11,1999-11-12,variagenics inc


In [9]:
identifiers['cleaned_tar_name']=identifiers['tar_name'].str.replace(',.+','').str.replace('\(.+?\)','')
identifiers['cleaned_tar_name']=identifiers.cleaned_tar_name.apply(''.join).str.replace('[^A-Za-z\s]+', ' ')
identifiers.head()

,anndate,anndate_3yrsago,tar_name,cleaned_tar_name
0,2001-07-02,1998-07-03,captec net lease realty inc,captec net lease realty inc
1,2001-01-10,1998-01-11,fritz cos inc,fritz cos inc
2,2001-01-16,1998-01-17,allaire corp,allaire corp
3,2001-01-22,1998-01-23,guest supply inc,guest supply inc
4,2001-01-23,1998-01-24,alliance bancorp,alliance bancorp


In [10]:
identifiers.set_index('cleaned_tar_name',inplace=True)
identifiers.loc['guest supply inc'][0]

Timestamp('2001-01-22 00:00:00')

In [11]:
 import pypatent

In [12]:
example = pypatent.Search(an='commercial bank of new york',get_patent_details=False).as_dataframe()
#example['patent_date']=pd.to_datetime(example['patent_date'])
example

AttributeError: 'NoneType' object has no attribute 'find_next'

In [ ]:
mask = (example['patent_date'] > identifiers.loc['guest supply inc'][1]) & (example['patent_date'] <= identifiers.loc['guest supply inc'][0])

In [ ]:
example[mask]

In [ ]:
relevant_patents = pd.DataFrame()
for tar in identifiers.index:
    print(tar)
    
    try:
        patent = pypatent.Search(an=tar,get_patent_details=False).as_dataframe()
        #mask = (patent['patent_date'] > identifiers.loc[tar][1]) & (patent['patent_date'] <= identifiers.loc[tar][0])
        #patent[mask]
        
        relevant_patents.append(patent)
    
    
    except:
        pass
    
relevant_patents.head(3)

captec net lease realty inc
fritz cos inc
allaire corp
guest supply inc
alliance bancorp
f m national
tci international inc
centura bank inc
dallas semiconductor corp
credit management solutions
wisconsin central transport
lamar capital corp
tosco corp
uti energy corp
egreetings network inc
women com networks
encal energy ltd
conectiv inc
commercial bank of new york
rgs energy group
imrglobal corp
oshman s sporting goods inc
spieker properties inc
american bancorporation inc
asi solutions inc
fourth shift corp
james river bankshares inc
